In [1]:
from env import get_db_url
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split



In [2]:
    conn = get_db_url('zillow')

    query = '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
            FROM properties_2017
            WHERE propertylandusetypeid = 261;  
            '''

    
    df = pd.read_sql(query, conn)

In [3]:
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
dtype: object

In [4]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [5]:
df =  df.dropna()

In [6]:
df = df.drop_duplicates(keep= False)


In [7]:
bin_cat = [0, 500000, 1000000, 1500000, 2000000, 5000000, 10000000, 25000000, 50000000, 75000000, 100000000]

df['value_cat'] = pd.cut(df['taxvaluedollarcnt'], bins=bin_cat, labels=False)

In [8]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,value_cat
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0,1
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0,1
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0,0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0,0
...,...,...,...,...,...,...,...,...
2152855,3.0,2.5,1809.0,405547.0,2012.0,4181.10,6059.0,0
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0,0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0,1
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0,1


## Acquire

In [9]:
def new_zillow_data():
   
    conn = get_db_url('zillow')

    query = '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
            FROM properties_2017
            WHERE propertylandusetypeid = 261;  
            '''

    
    df = pd.read_sql(query, conn)
    return df

In [10]:
def get_zillow_data():
    if os.path.isfile('zillow_df.csv'):
        df = pd.read_csv('zillow_df.csv', index_col = 0)

    else:

        df = new_zillow_data()

        df.to_csv('zillow_df.csv')
        return df

## Prep

##### Split

In [11]:
def split_zillow_data(df):
  
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)

    
    return train, validate, test


#### clean

In [12]:
def prep_zillow_data(df):
        df.replace(r'^\s*$', np.nan, regex=True)
        df.dropna()
        
        bin_cat = [0, 500000, 1000000, 1500000, 2000000, 5000000, 10000000, 25000000, 50000000, 75000000, 100000000]
        df['value_cat'] = pd.cut(df['taxvaluedollarcnt'], bins=bin_cat, labels=False)
    
        train, validate, test = split_zillow_data(df)
    
        return train, validate, test


### all in one

In [13]:
def wrangle_zillow():
    df = new_zillow_data()
    train, validate, test = prep_zillow_data(df)
    return train, validate, test

### trying it out

In [14]:
df = new_zillow_data()

In [15]:
df


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [16]:
train, validate, test = prep_zillow_data(df)

In [17]:
train

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,value_cat
1482948,2.0,1.0,924.0,52401.0,1924.0,928.68,6059.0,0.0
899718,3.0,2.0,1685.0,744547.0,1964.0,8578.20,6037.0,1.0
1108142,3.0,3.0,2210.0,524425.0,1965.0,5254.68,6059.0,1.0
1548945,3.0,2.5,2565.0,852000.0,1992.0,9668.54,6059.0,1.0
1243353,3.0,2.0,1108.0,205260.0,1941.0,3427.26,6037.0,0.0
...,...,...,...,...,...,...,...,...
1164599,3.0,3.0,1998.0,564000.0,1964.0,6897.62,6037.0,1.0
841555,3.0,2.0,1264.0,63557.0,1954.0,871.49,6037.0,0.0
943078,4.0,3.0,2403.0,172894.0,1961.0,2653.46,6037.0,0.0
1486320,3.0,2.0,1443.0,65362.0,1966.0,728.96,6111.0,0.0


In [ ]:
df = wrangle_zillow()

In [ ]:
df